# 📊 ПАЙПЛАЙН: ОФФЛАЙН И ОНЛАЙН ОБРАБОТКА

## Архитектура системы

### 🔄 ОФФЛАЙН ПАЙПЛАЙН (периодическая обработка)
Выполняется заранее (например, раз в день/неделю) и сохраняет предобработанные данные:

1. **Загрузка сырых данных**
   - События из всех каналов (`load_all_events`)
   - Данные о пользователях, товарах, брендах

2. **Предобработка и создание признаков**
   - Объединение событий с данными о товарах
   - Расчет агрегированных признаков для всех пользователей (`create_user_features`)
   - Временные признаки (datetime, hour, day_of_week)

3. **Сохранение результатов**
   - `user_features.parquet` - таблица признаков всех пользователей
   - `events_merged.parquet` - объединенные события (опционально, для детального анализа)
   - `metadata.json` - метаданные о времени обработки, версии данных

### ⚡ ОНЛАЙН ПАЙПЛАЙН (генерация по запросу)
Выполняется в реальном времени при запросе портрета пользователя:

1. **Загрузка предобработанных данных**
   - Загрузка признаков пользователя из `user_features.parquet`
   - Загрузка базовой информации о пользователе

2. **Генерация портрета**
   - Формирование портрета на основе предобработанных признаков
   - Дополнительные расчеты (если нужны свежие данные)

3. **Генерация рекомендаций**
   - Расчет скоров продуктов на основе портрета
   - Ранжирование и возврат топ-N рекомендаций


## 🔄 ОФФЛАЙН ПАЙПЛАЙН

Выполняется периодически (например, раз в день) для предобработки данных.


In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import glob
import warnings
import json
from datetime import datetime

warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [ ]:
def load_parquet_without_embeddings(file_path):
    try:
        parquet_file = pq.ParquetFile(file_path)
        columns = [col for col in parquet_file.schema_arrow.names if col != 'embedding']
        table = pq.read_table(file_path, columns=columns)
        return table.to_pandas()
    except Exception as e:
        print(f"Ошибка при загрузке {file_path}: {e}")
        return None

def load_all_events(base_path="./t_ecd_data/dataset/small", channels=None, sample_users=None):
    """
    Загружает все события из всех каналов один раз.
    Оптимизированная версия для пакетной обработки.
    
    :param base_path: Базовый путь к данным
    :param channels: Список каналов для загрузки (None = все каналы)
    :param sample_users: Список user_id для фильтрации (None = все пользователи)
    :return: DataFrame со всеми событиями
    """
    if channels is None:
        channels = ['marketplace', 'retail', 'offers']
    
    all_events = []
    
    print("📥 Загрузка всех событий...")
    
    for channel in channels:
        events_path = f"{base_path}/{channel}/events"
        event_files = sorted(glob.glob(f"{events_path}/*.pq"))
        
        print(f"  Загрузка канала {channel} ({len(event_files)} файлов)...")
        
        for file_path in tqdm(event_files, desc=f"  {channel}", leave=False):
            try:
                df = load_parquet_without_embeddings(file_path)
                if df is None:
                    continue
                
                if sample_users is not None:
                    df = df[df['user_id'].isin(sample_users)].copy()
                
                if len(df) > 0:
                    if 'timestamp' in df.columns:
                        base_date = datetime(2020, 1, 1)
                        df['datetime'] = base_date + pd.to_timedelta(df['timestamp'], unit='s')
                        df['date'] = df['datetime'].dt.date
                        df['hour'] = df['datetime'].dt.hour
                        df['day_of_week'] = df['datetime'].dt.dayofweek
                    
                    df['channel'] = channel
                    all_events.append(df)
                    
            except Exception as e:
                print(f"Ошибка при загрузке {file_path}: {e}")
                continue
    
    if all_events:
        print("🔗 Объединение событий...")
        combined = pd.concat(all_events, ignore_index=True)
        print(f"✅ Загружено {len(combined):,} событий")
        return combined.sort_values(['user_id', 'datetime'])
    return pd.DataFrame()


In [ ]:
def _initialize_empty_features(user_base):
    """
    Инициализирует пустые признаки для пользователей без событий.
    """
    empty_features = user_base.copy()
    
    # Числовые признаки
    numeric_cols = [
        'total_events', 'activity_days', 'events_per_day',
        'view_count', 'click_count', 'purchase_count',
        'view_to_click_rate', 'click_to_purchase_rate', 'purchase_rate',
        'total_spent', 'avg_purchase', 'std_purchase', 'purchase_count',
        'unique_categories', 'unique_brands', 'unique_channels',
        'avg_hour', 'hour_std', 'night_activity_ratio',
        'avg_price_interest', 'price_std', 'min_price_interest', 'max_price_interest', 'price_range'
    ]
    
    for col in numeric_cols:
        empty_features[col] = 0
    
    bool_cols = ['is_multi_channel']
    for col in bool_cols:
        empty_features[col] = False
    
    categorical_cols = ['preferred_channel', 'top_category']
    for col in categorical_cols:
        empty_features[col] = 'unknown'
    
    # Временные признаки
    empty_features['first_event'] = pd.NaT
    empty_features['last_event'] = pd.NaT
    
    return empty_features


def create_user_features(events_df, users_df, items_dict):
    """
    Создает ключевые признаки для пользователей на основе их поведения.
    
    Фокусируется на наиболее важных метриках для повышения производительности.
    
    :param events_df: DataFrame с событиями
    :param users_df: DataFrame с пользователями
    :param items_dict: Словарь с данными о товарах по каналам
    :return: DataFrame с признаками пользователей
    """
    
    user_base = users_df.copy()
    
    if len(events_df) == 0:
        return _initialize_empty_features(user_base)
    
    events_sorted = events_df.sort_values(['user_id', 'datetime']).copy()
    
    # ========== 1. БАЗОВАЯ СТАТИСТИКА ПО ВСЕМ СОБЫТИЯМ ==========
    all_events_stats = events_sorted.groupby('user_id').agg({
        'item_id': 'count',  # Общее количество событий
        'datetime': ['min', 'max'],  # Первое и последнее событие
    }).reset_index()
    all_events_stats.columns = ['user_id', 'total_events', 'first_event', 'last_event']
    
    # Период активности
    all_events_stats['activity_days'] = (
        (all_events_stats['last_event'] - all_events_stats['first_event']).dt.total_seconds() / 86400
    ).fillna(0)
    all_events_stats['events_per_day'] = (
        all_events_stats['total_events'] / (all_events_stats['activity_days'] + 1)
    )
    
    # ========== 2. ВОРОНКА КОНВЕРСИИ ==========
    funnel_stats = events_sorted.groupby(['user_id', 'action_type']).size().unstack(fill_value=0)
    funnel_stats = funnel_stats.reset_index()
    
    # Вычисляем конверсии
    if 'view' in funnel_stats.columns:
        funnel_stats['view_count'] = funnel_stats['view']
    else:
        funnel_stats['view_count'] = 0
    
    if 'click' in funnel_stats.columns:
        funnel_stats['click_count'] = funnel_stats['click']
    else:
        funnel_stats['click_count'] = 0
    
    if 'clickout' in funnel_stats.columns:
        funnel_stats['purchase_count'] = funnel_stats['clickout']
    else:
        funnel_stats['purchase_count'] = 0
    
    # Объединяем с базовой статистикой для получения total_events
    funnel_stats = funnel_stats.merge(all_events_stats[['user_id', 'total_events']], on='user_id', how='left')
    
    # Ключевые конверсии
    funnel_stats['view_to_click_rate'] = (
        funnel_stats['click_count'] / (funnel_stats['view_count'] + 1)
    )
    funnel_stats['click_to_purchase_rate'] = (
        funnel_stats['purchase_count'] / (funnel_stats['click_count'] + 1)
    )
    funnel_stats['purchase_rate'] = (
        funnel_stats['purchase_count'] / (funnel_stats['total_events'] + 1)
    )
    
    # ========== 3. СТАТИСТИКА ПО ПОКУПКАМ ==========
    purchases = events_sorted[events_sorted['action_type'] == 'clickout'].copy()
    
    if len(purchases) > 0:
        purchases_with_price = purchases[purchases['price'].notna()].copy()
        
        if len(purchases_with_price) > 0:
            purchase_stats = purchases_with_price.groupby('user_id').agg({
                'price': ['sum', 'mean', 'std'],
                'item_id': 'count'
            }).reset_index()
            purchase_stats.columns = [
                'user_id', 'total_spent', 'avg_purchase', 'std_purchase', 'purchase_count'
            ]
            purchase_stats['std_purchase'] = purchase_stats['std_purchase'].fillna(0)
        else:
            purchase_stats = pd.DataFrame({'user_id': []})
    else:
        purchase_stats = pd.DataFrame({'user_id': []})
    
    # ========== 4. КЛЮЧЕВЫЕ ПРИЗНАКИ РАЗНООБРАЗИЯ ==========
    # Используем векторизованные операции для ускорения
    diversity_stats = events_sorted.groupby('user_id').agg({
        'category': lambda x: x.nunique() if 'category' in events_sorted.columns else 0,
        'brand_id': lambda x: x.nunique() if 'brand_id' in events_sorted.columns else 0,
        'channel': lambda x: x.nunique() if 'channel' in events_sorted.columns else 0,
    }).reset_index()
    diversity_stats.columns = ['user_id', 'unique_categories', 'unique_brands', 'unique_channels']
    
    # Мультиканальность
    diversity_stats['is_multi_channel'] = diversity_stats['unique_channels'] > 1
    
    # Предпочитаемый канал
    if 'channel' in events_sorted.columns:
        preferred_channels = events_sorted.groupby('user_id')['channel'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'unknown').reset_index()
        preferred_channels.columns = ['user_id', 'preferred_channel']
        diversity_stats = diversity_stats.merge(preferred_channels, on='user_id', how='left')
    else:
        diversity_stats['preferred_channel'] = 'unknown'
    
    # ========== 5. ВРЕМЕННЫЕ ПРИЗНАКИ ==========
    if 'hour' in events_sorted.columns:
        temporal_stats = events_sorted.groupby('user_id')['hour'].agg(['mean', 'std']).reset_index()
        temporal_stats.columns = ['user_id', 'avg_hour', 'hour_std']
        temporal_stats['hour_std'] = temporal_stats['hour_std'].fillna(0)
        
        # Ночной покупатель (активность с 22:00 до 6:00)
        night_activity = events_sorted.groupby('user_id')['hour'].apply(
            lambda x: ((x < 6) | (x >= 22)).sum() / len(x)
        ).reset_index()
        night_activity.columns = ['user_id', 'night_activity_ratio']
        temporal_stats = temporal_stats.merge(night_activity, on='user_id', how='left')
    else:
        temporal_stats = pd.DataFrame({
            'user_id': events_sorted['user_id'].unique(),
            'avg_hour': 12,
            'hour_std': 0,
            'night_activity_ratio': 0
        })
    
    # ========== 6. ЦЕНОВЫЕ ПРИЗНАКИ ==========
    if 'price' in events_sorted.columns:
        price_events = events_sorted[events_sorted['price'].notna()]
        if len(price_events) > 0:
            price_stats = price_events.groupby('user_id')['price'].agg(['mean', 'std', 'min', 'max']).reset_index()
            price_stats.columns = ['user_id', 'avg_price_interest', 'price_std', 'min_price_interest', 'max_price_interest']
            price_stats['price_std'] = price_stats['price_std'].fillna(0)
            price_stats['price_range'] = price_stats['max_price_interest'] - price_stats['min_price_interest']
        else:
            price_stats = pd.DataFrame({'user_id': []})
    else:
        price_stats = pd.DataFrame({'user_id': []})
    
    # ========== 7. ТОП КАТЕГОРИЯ ==========
    if 'category' in events_sorted.columns:
        top_categories = events_sorted[events_sorted['category'].notna()].groupby(
            ['user_id', 'category']
        ).size().reset_index(name='category_count')
        
        if len(top_categories) > 0:
            top_category = top_categories.loc[top_categories.groupby('user_id')['category_count'].idxmax()]
            top_category = top_category[['user_id', 'category']].rename(columns={'category': 'top_category'})
        else:
            top_category = pd.DataFrame({'user_id': [], 'top_category': []})
    else:
        top_category = pd.DataFrame({'user_id': [], 'top_category': []})
    
    # ========== 8. ОБЪЕДИНЕНИЕ ВСЕХ ПРИЗНАКОВ ==========
    # Начинаем с базовых пользователей
    user_features_df = user_base.copy()
    
    # Объединяем все признаки
    user_features_df = user_features_df.merge(all_events_stats, on='user_id', how='left')
    user_features_df = user_features_df.merge(
        funnel_stats[['user_id', 'view_count', 'click_count', 'purchase_count',
                      'view_to_click_rate', 'click_to_purchase_rate', 'purchase_rate']],
        on='user_id', how='left'
    )
    
    if len(purchase_stats) > 0:
        user_features_df = user_features_df.merge(purchase_stats, on='user_id', how='left')
    
    user_features_df = user_features_df.merge(diversity_stats, on='user_id', how='left')
    user_features_df = user_features_df.merge(temporal_stats, on='user_id', how='left')
    
    if len(price_stats) > 0:
        user_features_df = user_features_df.merge(price_stats, on='user_id', how='left')
    
    if len(top_category) > 0:
        user_features_df = user_features_df.merge(top_category, on='user_id', how='left')
    
    # ========== 9. ЗАПОЛНЕНИЕ ПУСТЫХ ЗНАЧЕНИЙ ==========
    # Числовые колонки
    numeric_cols = [
        'total_events', 'activity_days', 'events_per_day',
        'view_count', 'click_count', 'purchase_count',
        'view_to_click_rate', 'click_to_purchase_rate', 'purchase_rate',
        'total_spent', 'avg_purchase', 'std_purchase', 'purchase_count',
        'unique_categories', 'unique_brands', 'unique_channels',
        'avg_hour', 'hour_std', 'night_activity_ratio',
        'avg_price_interest', 'price_std', 'min_price_interest', 'max_price_interest', 'price_range'
    ]
    
    for col in numeric_cols:
        if col in user_features_df.columns:
            user_features_df[col] = user_features_df[col].fillna(0)
    
    # Булевы колонки
    bool_cols = ['is_multi_channel']
    for col in bool_cols:
        if col in user_features_df.columns:
            user_features_df[col] = user_features_df[col].fillna(False)
    
    # Категориальные колонки
    categorical_cols = ['preferred_channel', 'top_category']
    for col in categorical_cols:
        if col in user_features_df.columns:
            user_features_df[col] = user_features_df[col].fillna('unknown')
    
    return user_features_df

def run_offline_pipeline(
    base_path="./t_ecd_data/dataset/small",
    output_dir="./preprocessed",
    channels=None,
    save_merged_events=False
):
    """
    Запускает оффлайн пайплайн для предобработки данных.
    
    Выполняет:
    1. Загрузку всех событий
    2. Объединение с данными о товарах
    3. Создание признаков пользователей
    4. Сохранение результатов
    
    :param base_path: Базовый путь к сырым данным
    :param output_dir: Директория для сохранения предобработанных данных
    :param channels: Список каналов для обработки (None = все)
    :param save_merged_events: Сохранять ли объединенные события (требует много места)
    :return: Словарь с путями к сохраненным файлам
    """
    import os
    from datetime import datetime
    
    print("="*60)
    print("🔄 ЗАПУСК ОФФЛАЙН ПАЙПЛАЙНА")
    print("="*60)
    print(f"Время начала: {datetime.now()}\n")
    
    # Создаем директорию для результатов
    os.makedirs(output_dir, exist_ok=True)
    
    # ========== 1. ЗАГРУЗКА СЫРЫХ ДАННЫХ ==========
    print("📥 ШАГ 1: Загрузка сырых данных...")
    
    # Загружаем справочники
    datasets = {}
    datasets['users'] = load_parquet_without_embeddings(f"{base_path}/users.pq")
    datasets['brands'] = load_parquet_without_embeddings(f"{base_path}/brands.pq")
    
    if channels is None:
        channels = ['marketplace', 'retail', 'offers']
    
    items_dict = {}
    for channel in channels:
        items_path = f"{base_path}/{channel}/items.pq"
        items_df = load_parquet_without_embeddings(items_path)
        datasets[f'{channel}_items'] = items_df
        items_dict[f'{channel}_items'] = items_df
    
    print(f"✅ Загружено справочников: {len(datasets)}")
    
    # Загружаем все события
    print("\n📥 Загрузка всех событий...")
    all_events = load_all_events(
        base_path=base_path,
        channels=channels
    )
    
    if len(all_events) == 0:
        print("❌ Ошибка: не удалось загрузить события")
        return None
    
    print(f"✅ Загружено {len(all_events):,} событий")
    
    # ========== 2. ОБЪЕДИНЕНИЕ С ДАННЫМИ О ТОВАРАХ ==========
    print("\n🔗 ШАГ 2: Объединение событий с данными о товарах...")
    
    # Объединяем события с товарами по каналам
    merged_events_list = []
    for channel in channels:
        channel_events = all_events[all_events['channel'] == channel].copy()
        items_df = items_dict.get(f'{channel}_items')
        
        if items_df is not None and len(channel_events) > 0:
            print(f"  Объединение канала {channel}...")
            channel_merged = channel_events.merge(
                items_df,
                on='item_id',
                how='left',
                suffixes=('', '_item')
            )
            merged_events_list.append(channel_merged)
        elif len(channel_events) > 0:
            merged_events_list.append(channel_events)
    
    if merged_events_list:
        events_merged = pd.concat(merged_events_list, ignore_index=True)
        print(f"✅ Объединено {len(events_merged):,} событий с товарами")
    else:
        events_merged = all_events
        print("⚠️  Использованы события без объединения")
    
    # ========== 3. СОЗДАНИЕ ПРИЗНАКОВ ПОЛЬЗОВАТЕЛЕЙ ==========
    print("\n📊 ШАГ 3: Создание признаков пользователей...")
    
    user_features_df = create_user_features(
        events_df=events_merged,
        users_df=datasets['users'],
        items_dict=items_dict
    )
    
    print(f"✅ Создано признаков для {len(user_features_df):,} пользователей")
    print(f"   Количество признаков: {len(user_features_df.columns)}")
    
    # ========== 4. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ==========
    print("\n💾 ШАГ 4: Сохранение результатов...")
    
    output_files = {}
    
    # Сохраняем признаки пользователей
    user_features_path = os.path.join(output_dir, "user_features.parquet")
    user_features_df.to_parquet(user_features_path, index=False, engine='pyarrow')
    output_files['user_features'] = user_features_path
    print(f"✅ Признаки пользователей сохранены: {user_features_path}")
    
    # Опционально: сохраняем объединенные события
    if save_merged_events:
        events_path = os.path.join(output_dir, "events_merged.parquet")
        events_merged.to_parquet(events_path, index=False, engine='pyarrow')
        output_files['events_merged'] = events_path
        print(f"✅ Объединенные события сохранены: {events_path}")
    
    # Сохраняем метаданные
    metadata = {
        'pipeline_version': '1.0',
        'processing_time': datetime.now().isoformat(),
        'base_path': base_path,
        'channels': channels,
        'total_users': len(user_features_df),
        'total_events': len(events_merged),
        'features_count': len(user_features_df.columns),
        'output_files': output_files
    }
    
    metadata_path = os.path.join(output_dir, "metadata.json")
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False, default=str)
    output_files['metadata'] = metadata_path
    print(f"✅ Метаданные сохранены: {metadata_path}")
    
    print("\n" + "="*60)
    print("✅ ОФФЛАЙН ПАЙПЛАЙН ЗАВЕРШЕН УСПЕШНО")
    print("="*60)
    print(f"Время завершения: {datetime.now()}")
    print(f"\nСохраненные файлы:")
    for key, path in output_files.items():
        file_size = os.path.getsize(path) / (1024 * 1024)  # MB
        print(f"  - {key}: {path} ({file_size:.2f} MB)")
    
    return output_files


## ⚡ ОНЛАЙН ПАЙПЛАЙН

Выполняется в реальном времени при запросе портрета пользователя.


In [4]:
def load_preprocessed_data(preprocessed_dir="./preprocessed"):
    """
    Загружает предобработанные данные из оффлайн пайплайна.
    
    :param preprocessed_dir: Директория с предобработанными данными
    :return: Словарь с загруженными данными
    """
    import os
    
    user_features_path = os.path.join(preprocessed_dir, "user_features.parquet")
    metadata_path = os.path.join(preprocessed_dir, "metadata.json")
    
    if not os.path.exists(user_features_path):
        raise FileNotFoundError(
            f"Файл признаков не найден: {user_features_path}\n"
            "Сначала запустите оффлайн пайплайн (run_offline_pipeline)"
        )
    
    print("📥 Загрузка предобработанных данных...")
    
    # Загружаем признаки пользователей
    user_features_df = pd.read_parquet(user_features_path, engine='pyarrow')
    print(f"✅ Загружено признаков для {len(user_features_df):,} пользователей")
    
    # Загружаем метаданные
    metadata = {}
    if os.path.exists(metadata_path):
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        print(f"✅ Загружены метаданные (версия: {metadata.get('pipeline_version', 'unknown')})")
    
    return {
        'user_features': user_features_df,
        'metadata': metadata
    }


def create_user_portrait_online(
    user_id,
    preprocessed_data,
    product_mapping=None
):
    """
    Создает портрет пользователя на основе предобработанных данных (ОНЛАЙН).
    
    Быстрая версия, использующая предобработанные признаки.
    
    :param user_id: ID пользователя
    :param preprocessed_data: Словарь с предобработанными данными (из load_preprocessed_data)
    :param product_mapping: Словарь с маппингом продуктов (опционально)
    :return: Словарь с портретом пользователя и рекомендациями
    """
    user_features_df = preprocessed_data['user_features']
    
    print(f"\n{'='*60}")
    print(f"⚡ ГЕНЕРАЦИЯ ПОРТРЕТА ПОЛЬЗОВАТЕЛЯ: {user_id}")
    print(f"{'='*60}")
    
    # Извлекаем признаки пользователя
    user_features = user_features_df[user_features_df['user_id'] == user_id]
    
    if len(user_features) == 0:
        print(f"❌ Пользователь {user_id} не найден в предобработанных данных")
        return None
    
    user_features = user_features.iloc[0]
    
    # Формируем портрет из предобработанных признаков
    portrait = {
        'user_id': int(user_id),
        'socdem_cluster': user_features.get('socdem_cluster'),
        'region': user_features.get('region'),
        
        # Базовая статистика
        'total_events': int(user_features.get('total_events', 0)),
        'first_event': user_features.get('first_event'),
        'last_event': user_features.get('last_event'),
        'activity_days': float(user_features.get('activity_days', 0)),
        'events_per_day': float(user_features.get('events_per_day', 0)),
        
        # Воронка конверсии
        'view_count': int(user_features.get('view_count', 0)),
        'click_count': int(user_features.get('click_count', 0)),
        'purchase_count': int(user_features.get('purchase_count', 0)),
        'view_to_click_rate': float(user_features.get('view_to_click_rate', 0)),
        'click_to_purchase_rate': float(user_features.get('click_to_purchase_rate', 0)),
        'purchase_rate': float(user_features.get('purchase_rate', 0)),
        
        # Финансовые показатели
        'total_spent': float(user_features.get('total_spent', 0)),
        'avg_purchase': float(user_features.get('avg_purchase', 0)),
        'std_purchase': float(user_features.get('std_purchase', 0)),
        'min_purchase': float(user_features.get('min_purchase', 0)) if 'min_purchase' in user_features else 0,
        'max_purchase': float(user_features.get('max_purchase', 0)) if 'max_purchase' in user_features else 0,
        
        # Разнообразие
        'unique_categories': int(user_features.get('unique_categories', 0)),
        'unique_brands': int(user_features.get('unique_brands', 0)),
        'unique_channels': int(user_features.get('unique_channels', 0)),
        'is_multi_channel': bool(user_features.get('is_multi_channel', False)),
        'preferred_channel': user_features.get('preferred_channel', 'unknown'),
        'top_category': user_features.get('top_category', 'unknown'),
        
        # Временные паттерны
        'avg_hour': float(user_features.get('avg_hour', 12)) if pd.notna(user_features.get('avg_hour')) else 12,
        'hour_std': float(user_features.get('hour_std', 0)),
        'night_activity_ratio': float(user_features.get('night_activity_ratio', 0)),
        
        # Ценовые предпочтения
        'avg_price_interest': float(user_features.get('avg_price_interest', 0)),
        'price_std': float(user_features.get('price_std', 0)),
        'min_price_interest': float(user_features.get('min_price_interest', 0)),
        'max_price_interest': float(user_features.get('max_price_interest', 0)),
        'price_range': float(user_features.get('price_range', 0)),
    }
    
    print("✅ Портрет создан на основе предобработанных признаков")
    
    # Генерируем рекомендации, если передан product_mapping
    recommendations = None
    
    return {
        'portrait': portrait,
        'recommendations': recommendations,
        'metadata': {
            'generated_at': datetime.now().isoformat(),
            'source': 'preprocessed_features'
        }
    }


def get_user_portrait_api(user_id, preprocessed_dir="./preprocessed", product_mapping=None):
    """
    API-функция для получения портрета пользователя (для использования в веб-сервисе).
    
    :param user_id: ID пользователя
    :param preprocessed_dir: Директория с предобработанными данными
    :param product_mapping: Словарь с маппингом продуктов (опционально)
    :return: Словарь с портретом и рекомендациями
    """
    # Загружаем предобработанные данные (можно кэшировать в продакшене)
    preprocessed_data = load_preprocessed_data(preprocessed_dir)
    
    # Генерируем портрет
    result = create_user_portrait_online(
        user_id=user_id,
        preprocessed_data=preprocessed_data,
        product_mapping=product_mapping
    )
    
    return result


## 📋 ИСПОЛЬЗОВАНИЕ ПАЙПЛАЙНОВ

### Пример 1: Запуск оффлайн пайплайна


In [5]:
# ============================================================================
# ПРИМЕР: ЗАПУСК ОФФЛАЙН ПАЙПЛАЙНА
# ============================================================================

# Запускаем оффлайн пайплайн (выполняется один раз, например, раз в день)
output_files = run_offline_pipeline(
    base_path="./t_ecd_data/dataset/small",
    output_dir="./preprocessed",
    channels=['marketplace', 'retail', 'offers'],
    save_merged_events=False  # Не сохраняем объединенные события (экономия места)
)


🔄 ЗАПУСК ОФФЛАЙН ПАЙПЛАЙНА
Время начала: 2025-11-28 18:37:11.041047

📥 ШАГ 1: Загрузка сырых данных...
✅ Загружено справочников: 5

📥 Загрузка всех событий...
📥 Загрузка всех событий...
  Загрузка канала marketplace (227 файлов)...


  Загрузка канала retail (227 файлов)...


  Загрузка канала offers (227 файлов)...


🔗 Объединение событий...


KeyboardInterrupt: 

### Пример 2: Генерация портрета пользователя онлайн


In [ ]:
# ============================================================================
# ПРИМЕР: ГЕНЕРАЦИЯ ПОРТРЕТА ПОЛЬЗОВАТЕЛЯ ОНЛАЙН
# ============================================================================

# Загружаем предобработанные данные (один раз при старте сервиса)
# preprocessed_data = load_preprocessed_data(preprocessed_dir="./preprocessed")

# Генерируем портрет для конкретного пользователя
# target_user_id = 25770580
# result = create_user_portrait_online(
#     user_id=target_user_id,
#     preprocessed_data=preprocessed_data,
#     product_mapping=product_mapping  # если доступен
# )

# if result:
#     print_user_portrait(result['portrait'])
#     if result['recommendations']:
#         print_recommendations(result['recommendations'], product_mapping)


## 📝 РЕЗЮМЕ АРХИТЕКТУРЫ

### Что происходит в ОФФЛАЙН пайплайне:

1. **Загрузка данных** (`load_all_events`)
   - Чтение всех событий из всех каналов
   - Загрузка справочников (users, items, brands)

2. **Предобработка** (`merge_user_events_with_items`)
   - Объединение событий с данными о товарах
   - Добавление временных признаков (datetime, hour, day_of_week)

3. **Создание признаков** (`create_user_features`)
   - Агрегация событий по пользователям
   - Расчет статистик (конверсии, покупки, разнообразие)
   - Временные и ценовые признаки

4. **Сохранение** (в `./preprocessed/`)
   - `user_features.parquet` - таблица признаков всех пользователей
   - `metadata.json` - метаданные о пайплайне
   - `events_merged.parquet` - опционально (если `save_merged_events=True`)

### Что происходит в ОНЛАЙН пайплайне:

1. **Загрузка предобработанных данных** (`load_preprocessed_data`)
   - Загрузка `user_features.parquet` (один раз при старте сервиса)
   - Загрузка метаданных

2. **Генерация портрета** (`create_user_portrait_online`)
   - Извлечение признаков конкретного пользователя
   - Формирование словаря портрета
   - **НЕ выполняет тяжелые вычисления** - все уже предобработано

3. **Генерация рекомендаций** (`calculate_product_recommendations`)
   - Расчет скоров продуктов на основе портрета
   - Ранжирование и возврат топ-N

### Ключевые файлы:

- **Оффлайн**: `run_offline_pipeline()` - запуск предобработки
- **Онлайн**: `get_user_portrait_api()` - API для получения портрета
- **Вспомогательные**: `load_preprocessed_data()`, `create_user_portrait_online()`


In [ ]:
def calculate_entropy(series):
    """
    Вычисляет энтропию Шеннона для серии.
    """
    value_counts = series.value_counts()
    probabilities = value_counts / len(series)
    return -np.sum(probabilities * np.log2(probabilities + 1e-10))


def get_user_events_from_preloaded(user_id, all_events_df):
    """
    Извлекает события конкретного пользователя из предзагруженного DataFrame.
    
    :param user_id: ID пользователя
    :param all_events_df: DataFrame со всеми событиями
    :return: DataFrame с событиями пользователя
    """
    if len(all_events_df) == 0:
        return pd.DataFrame()
    
    user_events = all_events_df[all_events_df['user_id'] == user_id].copy()
    return user_events.sort_values('datetime')


def merge_user_events_with_items(user_events, items_dict):
    """
    Объединяет события пользователя с данными о товарах.
    
    :param user_events: DataFrame с событиями пользователя
    :param items_dict: Словарь с данными о товарах по каналам
    :return: Объединенный DataFrame
    """
    if len(user_events) == 0:
        return user_events
    
    merged_events = []
    
    for channel in user_events['channel'].unique():
        channel_events = user_events[user_events['channel'] == channel].copy()
        items_df = items_dict.get(f'{channel}_items')
        
        if items_df is not None:
            channel_merged = channel_events.merge(
                items_df,
                on='item_id',
                how='left',
                suffixes=('', '_item')
            )
            merged_events.append(channel_merged)
        else:
            merged_events.append(channel_events)
    
    if merged_events:
        return pd.concat(merged_events, ignore_index=True)
    return pd.DataFrame()


def create_user_portrait(user_id, users_df, items_dict, base_path="./t_ecd_data/dataset/small", 
                         preloaded_events=None):
    """
    Создает полный портрет пользователя для конкретного user_id.
    
    :param user_id: ID пользователя
    :param users_df: DataFrame с пользователями
    :param items_dict: Словарь с данными о товарах по каналам
    :param base_path: Базовый путь к данным (используется только если preloaded_events=None)
    :param preloaded_events: Предзагруженный DataFrame со всеми событиями (опционально)
    :return: Словарь с портретом пользователя
    """
    print(f"\n{'='*60}")
    print(f"Создание портрета пользователя: {user_id}")
    print(f"{'='*60}")
    
    # Получаем базовую информацию о пользователе
    user_info = users_df[users_df['user_id'] == user_id]
    if len(user_info) == 0:
        return None
    
    user_info = user_info.iloc[0]
    portrait = {
        'user_id': user_id,
        'socdem_cluster': user_info.get('socdem_cluster'),
        'region': user_info.get('region')
    }
    
    # Загружаем события пользователя
    if preloaded_events is not None:
        print("📥 Извлечение событий пользователя из предзагруженных данных...")
        user_events = get_user_events_from_preloaded(user_id, preloaded_events)
    else:
        print("📥 Загрузка событий пользователя...")
        user_events = load_user_events(user_id, base_path)
    
    if len(user_events) == 0:
        print("⚠️  Пользователь не имеет событий")
        return portrait
    
    print(f"✅ Найдено {len(user_events):,} событий")
    
    # Объединяем с данными о товарах
    print("🔗 Объединение с данными о товарах...")
    user_events = merge_user_events_with_items(user_events, items_dict)
    print(f"✅ Объединено {len(user_events):,} событий с товарами")
    
    # ========== БАЗОВАЯ СТАТИСТИКА ==========
    print("📊 Расчет базовой статистики...")
    portrait['total_events'] = len(user_events)
    portrait['first_event'] = user_events['datetime'].min() if 'datetime' in user_events.columns else None
    portrait['last_event'] = user_events['datetime'].max() if 'datetime' in user_events.columns else None
    
    if portrait['first_event'] and portrait['last_event']:
        portrait['activity_days'] = (portrait['last_event'] - portrait['first_event']).days + 1
        portrait['events_per_day'] = portrait['total_events'] / portrait['activity_days']
    else:
        portrait['activity_days'] = 0
        portrait['events_per_day'] = 0
    
    # ========== ВОРОНКА КОНВЕРСИИ ==========
    print("🔄 Анализ воронки конверсии...")
    if 'action_type' in user_events.columns:
        action_counts = user_events['action_type'].value_counts()
        portrait['view_count'] = action_counts.get('view', 0)
        portrait['click_count'] = action_counts.get('click', 0)
        portrait['purchase_count'] = action_counts.get('clickout', 0)
        portrait['like_count'] = action_counts.get('like', 0)
        
        portrait['view_to_click_rate'] = (
            portrait['click_count'] / (portrait['view_count'] + 1)
        )
        portrait['click_to_purchase_rate'] = (
            portrait['purchase_count'] / (portrait['click_count'] + 1)
        )
        portrait['purchase_rate'] = (
            portrait['purchase_count'] / (portrait['total_events'] + 1)
        )
    
    # ========== СТАТИСТИКА ПО ПОКУПКАМ ==========
    print("💰 Анализ покупок...")
    purchases = user_events[user_events['action_type'] == 'clickout'].copy() if 'action_type' in user_events.columns else pd.DataFrame()
    
    if len(purchases) > 0 and 'price' in purchases.columns:
        purchases_with_price = purchases[purchases['price'].notna()]
        if len(purchases_with_price) > 0:
            portrait['total_spent'] = purchases_with_price['price'].sum()
            portrait['avg_purchase'] = purchases_with_price['price'].mean()
            portrait['std_purchase'] = purchases_with_price['price'].std()
            portrait['min_purchase'] = purchases_with_price['price'].min()
            portrait['max_purchase'] = purchases_with_price['price'].max()
        else:
            portrait['total_spent'] = 0
            portrait['avg_purchase'] = 0
            portrait['std_purchase'] = 0
            portrait['min_purchase'] = 0
            portrait['max_purchase'] = 0
    else:
        portrait['total_spent'] = 0
        portrait['avg_purchase'] = 0
        portrait['std_purchase'] = 0
        portrait['min_purchase'] = 0
        portrait['max_purchase'] = 0
    
    # ========== РАЗНООБРАЗИЕ ==========
    print("🎯 Анализ разнообразия...")
    if 'category' in user_events.columns:
        portrait['unique_categories'] = user_events['category'].nunique()
        top_category = user_events['category'].value_counts().head(1)
        portrait['top_category'] = top_category.index[0] if len(top_category) > 0 else None
        portrait['top_category_count'] = top_category.iloc[0] if len(top_category) > 0 else 0
    else:
        portrait['unique_categories'] = 0
        portrait['top_category'] = None
        portrait['top_category_count'] = 0
    
    if 'brand_id' in user_events.columns:
        portrait['unique_brands'] = user_events['brand_id'].nunique()
        top_brand = user_events['brand_id'].value_counts().head(1)
        portrait['top_brand_id'] = top_brand.index[0] if len(top_brand) > 0 else None
    else:
        portrait['unique_brands'] = 0
        portrait['top_brand_id'] = None
    
    if 'channel' in user_events.columns:
        portrait['unique_channels'] = user_events['channel'].nunique()
        portrait['is_multi_channel'] = portrait['unique_channels'] > 1
        preferred_channel = user_events['channel'].value_counts().head(1)
        portrait['preferred_channel'] = preferred_channel.index[0] if len(preferred_channel) > 0 else None
    else:
        portrait['unique_channels'] = 0
        portrait['is_multi_channel'] = False
        portrait['preferred_channel'] = None
    
    # ========== ВРЕМЕННЫЕ ПРИЗНАКИ ==========
    print("⏰ Анализ временных паттернов...")
    if 'hour' in user_events.columns:
        portrait['avg_hour'] = user_events['hour'].mean()
        portrait['hour_std'] = user_events['hour'].std()
        portrait['night_activity_ratio'] = (
            ((user_events['hour'] < 6) | (user_events['hour'] >= 22)).sum() / len(user_events)
        )
        portrait['preferred_hour'] = user_events['hour'].mode().iloc[0] if len(user_events['hour'].mode()) > 0 else None
    else:
        portrait['avg_hour'] = None
        portrait['hour_std'] = None
        portrait['night_activity_ratio'] = 0
        portrait['preferred_hour'] = None
    
    if 'day_of_week' in user_events.columns:
        portrait['preferred_day'] = user_events['day_of_week'].mode().iloc[0] if len(user_events['day_of_week'].mode()) > 0 else None
        portrait['weekend_activity_ratio'] = (
            (user_events['day_of_week'] >= 5).sum() / len(user_events)
        )
    else:
        portrait['preferred_day'] = None
        portrait['weekend_activity_ratio'] = 0
    
    # ========== ЦЕНОВЫЕ ПРИЗНАКИ ==========
    print("💵 Анализ ценовых предпочтений...")
    if 'price' in user_events.columns:
        price_events = user_events[user_events['price'].notna()]
        if len(price_events) > 0:
            portrait['avg_price_interest'] = price_events['price'].mean()
            portrait['price_std'] = price_events['price'].std()
            portrait['min_price_interest'] = price_events['price'].min()
            portrait['max_price_interest'] = price_events['price'].max()
            portrait['price_range'] = portrait['max_price_interest'] - portrait['min_price_interest']
        else:
            portrait['avg_price_interest'] = 0
            portrait['price_std'] = 0
            portrait['min_price_interest'] = 0
            portrait['max_price_interest'] = 0
            portrait['price_range'] = 0
    else:
        portrait['avg_price_interest'] = 0
        portrait['price_std'] = 0
        portrait['min_price_interest'] = 0
        portrait['max_price_interest'] = 0
        portrait['price_range'] = 0
    
    # ========== ДОПОЛНИТЕЛЬНЫЕ ПРИЗНАКИ ==========
    print("📈 Расчет дополнительных признаков...")
    if 'os' in user_events.columns:
        portrait['preferred_os'] = user_events['os'].mode().iloc[0] if len(user_events['os'].mode()) > 0 else None
    else:
        portrait['preferred_os'] = None
    
    if 'subdomain' in user_events.columns:
        portrait['preferred_subdomain'] = user_events['subdomain'].mode().iloc[0] if len(user_events['subdomain'].mode()) > 0 else None
    else:
        portrait['preferred_subdomain'] = None
    
    print("✅ Портрет пользователя создан!")
    return portrait


def print_user_portrait(portrait):
    """
    Красиво выводит портрет пользователя.
    
    :param portrait: Словарь с портретом пользователя
    """
    if portrait is None:
        print("❌ Портрет не найден")
        return
    
    print(f"\n{'='*60}")
    print(f"👤 ПОРТРЕТ ПОЛЬЗОВАТЕЛЯ: {portrait['user_id']}")
    print(f"{'='*60}")
    
    print(f"\n📋 БАЗОВАЯ ИНФОРМАЦИЯ:")
    print(f"  Социально-демографический кластер: {portrait.get('socdem_cluster', 'N/A')}")
    print(f"  Регион: {portrait.get('region', 'N/A')}")
    
    print(f"\n📊 АКТИВНОСТЬ:")
    print(f"  Всего событий: {portrait.get('total_events', 0):,}")
    print(f"  Первое событие: {portrait.get('first_event', 'N/A')}")
    print(f"  Последнее событие: {portrait.get('last_event', 'N/A')}")
    print(f"  Дней активности: {portrait.get('activity_days', 0)}")
    print(f"  Событий в день: {portrait.get('events_per_day', 0):.2f}")
    
    print(f"\n🔄 ВОРОНКА КОНВЕРСИИ:")
    print(f"  Просмотров: {portrait.get('view_count', 0):,}")
    print(f"  Кликов: {portrait.get('click_count', 0):,}")
    print(f"  Покупок: {portrait.get('purchase_count', 0):,}")
    print(f"  Лайков: {portrait.get('like_count', 0):,}")
    print(f"  Конверсия view→click: {portrait.get('view_to_click_rate', 0):.4f}")
    print(f"  Конверсия click→purchase: {portrait.get('click_to_purchase_rate', 0):.4f}")
    print(f"  Общая конверсия: {portrait.get('purchase_rate', 0):.4f}")
    
    print(f"\n💰 ФИНАНСОВЫЕ ПОКАЗАТЕЛИ:")
    print(f"  Общие траты: {portrait.get('total_spent', 0):.2f}")
    print(f"  Средний чек: {portrait.get('avg_purchase', 0):.2f}")
    print(f"  Мин. покупка: {portrait.get('min_purchase', 0):.2f}")
    print(f"  Макс. покупка: {portrait.get('max_purchase', 0):.2f}")
    print(f"  Стд. отклонение: {portrait.get('std_purchase', 0):.2f}")
    
    print(f"\n🎯 РАЗНООБРАЗИЕ:")
    print(f"  Уникальных категорий: {portrait.get('unique_categories', 0)}")
    print(f"  Топ категория: {portrait.get('top_category', 'N/A')}")
    print(f"  Уникальных брендов: {portrait.get('unique_brands', 0)}")
    print(f"  Уникальных каналов: {portrait.get('unique_channels', 0)}")
    print(f"  Мультиканальность: {'Да' if portrait.get('is_multi_channel', False) else 'Нет'}")
    print(f"  Предпочитаемый канал: {portrait.get('preferred_channel', 'N/A')}")
    
    print(f"\n⏰ ВРЕМЕННЫЕ ПАТТЕРНЫ:")
    print(f"  Средний час активности: {portrait.get('avg_hour', 'N/A')}")
    print(f"  Предпочитаемый час: {portrait.get('preferred_hour', 'N/A')}")
    print(f"  Ночная активность: {portrait.get('night_activity_ratio', 0):.2%}")
    print(f"  Предпочитаемый день: {portrait.get('preferred_day', 'N/A')}")
    print(f"  Активность в выходные: {portrait.get('weekend_activity_ratio', 0):.2%}")
    
    print(f"\n💵 ЦЕНОВЫЕ ПРЕДПОЧТЕНИЯ:")
    print(f"  Средний интерес к цене: {portrait.get('avg_price_interest', 0):.2f}")
    print(f"  Диапазон цен: {portrait.get('price_range', 0):.2f}")
    
    print(f"\n🔧 ТЕХНИЧЕСКИЕ:")
    print(f"  Предпочитаемая ОС: {portrait.get('preferred_os', 'N/A')}")
    print(f"  Предпочитаемый поддомен: {portrait.get('preferred_subdomain', 'N/A')}")
    
    print(f"\n{'='*60}\n")


In [14]:
def load_parquet_without_embeddings(file_path):
    try:
        parquet_file = pq.ParquetFile(file_path)
        columns = [col for col in parquet_file.schema_arrow.names if col != 'embedding']
        table = pq.read_table(file_path, columns=columns)
        return table.to_pandas()
    except Exception as e:
        print(f"Ошибка при загрузке {file_path}: {e}")
        return None

base_path = "./t_ecd_data/dataset/small"

datasets = {}

datasets['users'] = load_parquet_without_embeddings(f"{base_path}/users.pq")
datasets['brands'] = load_parquet_without_embeddings(f"{base_path}/brands.pq")

channels = ['marketplace', 'retail', 'offers']
for channel in channels:
    datasets[f'{channel}_items'] = load_parquet_without_embeddings(f"{base_path}/{channel}/items.pq")

print(f"Загружено датасетов: {len(datasets)}")

Загружено датасетов: 5


In [23]:
items_dict = {
    'marketplace_items': datasets['marketplace_items'],
    'retail_items': datasets['retail_items'],
    'offers_items': datasets['offers_items']
}

all_events = load_all_events(
    base_path="./t_ecd_data/dataset/small",
)

📥 Загрузка всех событий...
  Загрузка канала marketplace (227 файлов)...


  Загрузка канала retail (227 файлов)...


  Загрузка канала offers (227 файлов)...


🔗 Объединение событий...


KeyboardInterrupt: 

In [ ]:
target_user_id = "83491148"

portrait = create_user_portrait(
    user_id=target_user_id,
    users_df=datasets['users'],
    items_dict=items_dict,
    base_path="./t_ecd_data/dataset/small",
    preloaded_events=all_events
)


if portrait:
    print_user_portrait(portrait)
    
    import json
    with open(f'user_portrait_{target_user_id}.json', 'w', encoding='utf-8') as f:
        portrait_json = {}
        for k, v in portrait.items():
            if isinstance(v, (pd.Timestamp, datetime)):
                portrait_json[k] = str(v)
            elif pd.isna(v):
                portrait_json[k] = None
            else:
                portrait_json[k] = v
        json.dump(portrait_json, f, indent=2, ensure_ascii=False, default=str)
    print(f"💾 Портрет сохранен в user_portrait_{target_user_id}.json")
else:
    print(f"❌ Пользователь {target_user_id} не найден")



Создание портрета пользователя: 83491148
❌ Пользователь 83491148 не найден


In [ ]:
# 4. Создайте портрет пользователя с использованием предзагруженных событий
portrait = create_user_portrait(
    user_id=target_user_id,
    users_df=datasets['users'],
    items_dict=items_dict,
    base_path="./t_ecd_data/dataset/small",
    preloaded_events=all_events  # Используем предзагруженные события
)

# 4. Выведите портрет
if portrait:
    print_user_portrait(portrait)
    
    # 5. Сохраните портрет в JSON (опционально)
    import json
    with open(f'user_portrait_{target_user_id}.json', 'w', encoding='utf-8') as f:
        # Конвертируем datetime в строки для JSON
        portrait_json = {}
        for k, v in portrait.items():
            if isinstance(v, (pd.Timestamp, datetime)):
                portrait_json[k] = str(v)
            elif pd.isna(v):
                portrait_json[k] = None
            else:
                portrait_json[k] = v
        json.dump(portrait_json, f, indent=2, ensure_ascii=False, default=str)
    print(f"💾 Портрет сохранен в user_portrait_{target_user_id}.json")
else:
    print(f"❌ Пользователь {target_user_id} не найден")


In [ ]:
# ============================================================================
# РЕКОМЕНДАЦИИ БАНКОВСКИХ ПРОДУКТОВ НА ОСНОВЕ ПОРТРЕТА
# ============================================================================

def calculate_product_recommendations(portrait, product_mapping):
    """
    Рассчитывает рекомендации банковских продуктов на основе портрета пользователя.
    
    :param portrait: Словарь с портретом пользователя
    :param product_mapping: Словарь с маппингом продуктов
    :return: Словарь с рекомендациями (продукт -> скор)
    """
    if portrait is None:
        return None
    
    recommendations = {}
    
    # Извлекаем признаки из портрета
    total_spent = portrait.get('total_spent', 0) or 0
    avg_purchase = portrait.get('avg_purchase', 0) or 0
    purchase_count = portrait.get('purchase_count', 0) or 0
    events_per_day = portrait.get('events_per_day', 0) or 0
    unique_brands = portrait.get('unique_brands', 0) or 0
    top_category = portrait.get('top_category', '')
    unique_categories = portrait.get('unique_categories', 0)
    is_multi_channel = portrait.get('is_multi_channel', False)
    socdem_cluster = portrait.get('socdem_cluster', 0) or 0
    purchase_rate = portrait.get('purchase_rate', 0) or 0
    
    # Рассчитываем скоры для каждого продукта
    for product_id, product_info in product_mapping.items():
        score = 0.0
        
        # 1. Соответствие категориям (вес: 0.3)
        if top_category and top_category in product_info.get('categories', []):
            score += 0.3
        elif unique_categories > 0:
            # Частичное соответствие
            matching = sum(1 for cat in product_info.get('categories', []) 
                          if cat == top_category)
            if matching > 0:
                score += 0.15
        
        # 2. Финансовые показатели (вес: 0.3)
        if avg_purchase >= product_info.get('min_avg_purchase', 0):
            score += 0.15
        elif avg_purchase >= product_info.get('min_avg_purchase', 0) * 0.5:
            score += 0.075
        
        if total_spent >= product_info.get('min_total_spent', 0):
            score += 0.15
        elif total_spent >= product_info.get('min_total_spent', 0) * 0.5:
            score += 0.075
        
        # 3. Паттерны покупок (вес: 0.2)
        purchase_freq = product_info.get('purchase_frequency')
        if purchase_freq == 'high' and events_per_day > 0.1:
            score += 0.1
        elif purchase_freq == 'regular' and 0.05 < events_per_day <= 0.1:
            score += 0.1
        
        if purchase_rate > 0.1:  # Более 10% событий - покупки
            score += 0.1
        
        # 4. Разнообразие и активность (вес: 0.1)
        if product_info.get('unique_brands') and unique_brands >= product_info['unique_brands']:
            score += 0.05
        
        if is_multi_channel:
            score += 0.05
        
        # 5. Дополнительные бонусы (вес: 0.1)
        if socdem_cluster > 15:  # Высокий соц-дем кластер
            score += 0.05
        
        if purchase_count > 10:  # Активный покупатель
            score += 0.05
        
        recommendations[product_id] = min(score, 1.0)  # Ограничиваем максимум 1.0
    
    # Сортируем по скору
    recommendations = dict(sorted(recommendations.items(), key=lambda x: x[1], reverse=True))
    return recommendations


def print_recommendations(recommendations, product_mapping):
    """
    Красиво выводит рекомендации банковских продуктов.
    
    :param recommendations: Словарь с рекомендациями (продукт -> скор)
    :param product_mapping: Словарь с маппингом продуктов
    """
    if recommendations is None:
        print("❌ Рекомендации не найдены")
        return
    
    product_names = {
        'credit_consumer': '💳 Потребительский кредит',
        'credit_mortgage': '🏠 Ипотека',
        'deposit_high_income': '💰 Вклад для высокого дохода',
        'deposit_savings': '💵 Накопительный вклад',
        'investment': '📈 Инвестиционные продукты',
        'insurance_health': '🏥 Страхование здоровья',
        'insurance_property': '🏘️ Страхование имущества',
        'insurance_travel': '✈️ Страхование путешествий',
        'premium_service': '👑 Премиальное обслуживание',
        'savings_account': '💼 Накопительный счет'
    }
    
    print(f"\n{'='*60}")
    print(f"💡 РЕКОМЕНДАЦИИ БАНКОВСКИХ ПРОДУКТОВ")
    print(f"{'='*60}\n")
    
    # Топ-5 рекомендаций
    top_5 = list(recommendations.items())[:5]
    
    for i, (product_id, score) in enumerate(top_5, 1):
        product_name = product_names.get(product_id, product_id)
        
        # Определяем уровень рекомендации
        if score >= 0.7:
            level = "🔥 Высокий приоритет"
        elif score >= 0.4:
            level = "⭐ Средний приоритет"
        else:
            level = "💡 Низкий приоритет"
        
        print(f"{i}. {product_name}")
        print(f"   Скор: {score:.3f} | {level}")
        
        # Показываем требования продукта
        product_info = product_mapping.get(product_id, {})
        requirements = []
        if product_info.get('min_avg_purchase'):
            requirements.append(f"Средний чек: {product_info['min_avg_purchase']:.0f}+")
        if product_info.get('min_total_spent'):
            requirements.append(f"Общие траты: {product_info['min_total_spent']:.0f}+")
        if requirements:
            print(f"   Требования: {', '.join(requirements)}")
        print()
    
    print(f"{'='*60}\n")


# Пример использования с портретом пользователя
if 'portrait' in locals() and portrait:
    # Загружаем маппинг продуктов (если еще не загружен)
    if 'product_mapping' not in locals():
        product_mapping, _ = create_product_mapping()
    
    # Рассчитываем рекомендации
    recommendations = calculate_product_recommendations(portrait, product_mapping)
    
    # Выводим рекомендации
    if recommendations:
        print_recommendations(recommendations, product_mapping)
        
        # Сохраняем рекомендации в JSON
        import json
        user_id = portrait['user_id']
        with open(f'user_recommendations_{user_id}.json', 'w', encoding='utf-8') as f:
            json.dump(recommendations, f, indent=2, ensure_ascii=False)
        print(f"💾 Рекомендации сохранены в user_recommendations_{user_id}.json")
else:
    print("⚠️  Сначала создайте портрет пользователя (ячейка выше)")


# 📚 ДОКУМЕНТАЦИЯ ПО ОПТИМИЗАЦИИ

## Оптимизация загрузки событий

### Проблема
Функция `load_user_events` загружает и фильтрует события для каждого пользователя отдельно, что приводит к многократному чтению одних и тех же файлов.

### Решение
Используйте функции `load_all_events` и `get_user_events_from_preloaded` для предзагрузки всех событий один раз.

### Примеры использования:

**Для одного пользователя:**
```python
# Предзагрузка событий
all_events = load_all_events(
    base_path="./t_ecd_data/dataset/small",
    sample_users=[user_id]  # Опционально: фильтрация по пользователям
)

# Создание портрета с предзагруженными событиями
portrait = create_user_portrait(
    user_id=user_id,
    users_df=datasets['users'],
    items_dict=items_dict,
    preloaded_events=all_events
)
```

**Для нескольких пользователей:**
```python
# Функция create_user_portraits_batch автоматически предзагружает события
portraits = create_user_portraits_batch(
    user_ids=[user1, user2, user3],
    users_df=datasets['users'],
    items_dict=items_dict,
    base_path="./t_ecd_data/dataset/small"
)
```

### Преимущества:
- ✅ События загружаются один раз вместо N раз (где N - количество пользователей)
- ✅ Значительное ускорение при обработке нескольких пользователей
- ✅ Меньше операций I/O с диском


In [7]:
# ============================================================================
# ОБНОВЛЕННАЯ ВЕРСИЯ create_user_portraits_batch с оптимизацией
# Скопируйте этот код в ячейку 5, заменив старую версию функции
# ============================================================================

def create_user_portraits_batch_optimized(user_ids, users_df, items_dict, base_path="./t_ecd_data/dataset/small", 
                                         save_to_file=True, output_dir=".", preloaded_events=None):
    """
    Создает портреты для нескольких пользователей (ОПТИМИЗИРОВАННАЯ ВЕРСИЯ).
    
    :param user_ids: Список ID пользователей
    :param users_df: DataFrame с пользователями
    :param items_dict: Словарь с данными о товарах по каналам
    :param base_path: Базовый путь к данным (используется только если preloaded_events=None)
    :param save_to_file: Сохранять ли результаты в файлы
    :param output_dir: Директория для сохранения результатов
    :param preloaded_events: Предзагруженный DataFrame со всеми событиями (опционально)
    :return: Список портретов пользователей
    """
    portraits = []
    
    print(f"🚀 Начало пакетной обработки {len(user_ids)} пользователей\n")
    
    # ОПТИМИЗАЦИЯ: Если события не предзагружены, загружаем их один раз
    if preloaded_events is None:
        print("📥 Предзагрузка событий для всех пользователей...")
        preloaded_events = load_all_events(base_path=base_path, sample_users=user_ids)
        print(f"✅ Предзагружено {len(preloaded_events):,} событий\n")
    
    for i, user_id in enumerate(user_ids, 1):
        print(f"\n[{i}/{len(user_ids)}] Обработка пользователя {user_id}...")
        
        try:
            portrait = create_user_portrait(
                user_id=user_id,
                users_df=users_df,
                items_dict=items_dict,
                base_path=base_path,
                preloaded_events=preloaded_events  # Используем предзагруженные события
            )
            
            if portrait:
                portraits.append(portrait)
                
                if save_to_file:
                    # Сохраняем портрет
                    import json
                    import os
                    os.makedirs(output_dir, exist_ok=True)
                    
                    portrait_json = {}
                    for k, v in portrait.items():
                        if isinstance(v, (pd.Timestamp, datetime)):
                            portrait_json[k] = str(v)
                        elif pd.isna(v):
                            portrait_json[k] = None
                        else:
                            portrait_json[k] = v
                    
                    filepath = os.path.join(output_dir, f'user_portrait_{user_id}.json')
                    with open(filepath, 'w', encoding='utf-8') as f:
                        json.dump(portrait_json, f, indent=2, ensure_ascii=False, default=str)
                    
                    # Сохраняем рекомендации, если доступен product_mapping
                    if 'product_mapping' in locals():
                        recommendations = calculate_product_recommendations(portrait, product_mapping)
                        if recommendations:
                            rec_filepath = os.path.join(output_dir, f'user_recommendations_{user_id}.json')
                            with open(rec_filepath, 'w', encoding='utf-8') as f:
                                json.dump(recommendations, f, indent=2, ensure_ascii=False)
            
        except Exception as e:
            print(f"❌ Ошибка при обработке пользователя {user_id}: {e}")
            continue
    
    print(f"\n✅ Пакетная обработка завершена!")
    print(f"   Обработано: {len(portraits)}/{len(user_ids)} пользователей")
    
    return portraits


In [8]:
# ============================================================================
# ПАКЕТНАЯ ОБРАБОТКА НЕСКОЛЬКИХ ПОЛЬЗОВАТЕЛЕЙ
# ============================================================================

def create_user_portraits_batch(user_ids, users_df, items_dict, base_path="./t_ecd_data/dataset/small", 
                                save_to_file=True, output_dir="."):
    """
    Создает портреты для нескольких пользователей.
    
    :param user_ids: Список ID пользователей
    :param users_df: DataFrame с пользователями
    :param items_dict: Словарь с данными о товарах по каналам
    :param base_path: Базовый путь к данным
    :param save_to_file: Сохранять ли результаты в файлы
    :param output_dir: Директория для сохранения результатов
    :return: Список портретов пользователей
    """
    portraits = []
    
    print(f"🚀 Начало пакетной обработки {len(user_ids)} пользователей\n")
    
    for i, user_id in enumerate(user_ids, 1):
        print(f"\n[{i}/{len(user_ids)}] Обработка пользователя {user_id}...")
        
        try:
            portrait = create_user_portrait(
                user_id=user_id,
                users_df=users_df,
                items_dict=items_dict,
                base_path=base_path
            )
            
            if portrait:
                portraits.append(portrait)
                
                if save_to_file:
                    # Сохраняем портрет
                    import json
                    import os
                    os.makedirs(output_dir, exist_ok=True)
                    
                    portrait_json = {}
                    for k, v in portrait.items():
                        if isinstance(v, (pd.Timestamp, datetime)):
                            portrait_json[k] = str(v)
                        elif pd.isna(v):
                            portrait_json[k] = None
                        else:
                            portrait_json[k] = v
                    
                    filepath = os.path.join(output_dir, f'user_portrait_{user_id}.json')
                    with open(filepath, 'w', encoding='utf-8') as f:
                        json.dump(portrait_json, f, indent=2, ensure_ascii=False, default=str)
                    
                    # Сохраняем рекомендации, если доступен product_mapping
                    if 'product_mapping' in locals():
                        recommendations = calculate_product_recommendations(portrait, product_mapping)
                        if recommendations:
                            rec_filepath = os.path.join(output_dir, f'user_recommendations_{user_id}.json')
                            with open(rec_filepath, 'w', encoding='utf-8') as f:
                                json.dump(recommendations, f, indent=2, ensure_ascii=False)
            
        except Exception as e:
            print(f"❌ Ошибка при обработке пользователя {user_id}: {e}")
            continue
    
    print(f"\n✅ Пакетная обработка завершена!")
    print(f"   Обработано: {len(portraits)}/{len(user_ids)} пользователей")
    
    return portraits


def create_portraits_summary(portraits, output_file="portraits_summary.json"):
    """
    Создает сводную статистику по портретам пользователей.
    
    :param portraits: Список портретов пользователей
    :param output_file: Имя файла для сохранения сводки
    """
    if not portraits:
        print("❌ Нет портретов для анализа")
        return
    
    summary = {
        'total_users': len(portraits),
        'statistics': {}
    }
    
    # Собираем статистику по числовым признакам
    numeric_fields = [
        'total_events', 'activity_days', 'events_per_day',
        'view_count', 'click_count', 'purchase_count',
        'total_spent', 'avg_purchase',
        'unique_categories', 'unique_brands', 'unique_channels',
        'avg_hour', 'night_activity_ratio'
    ]
    
    for field in numeric_fields:
        values = [p.get(field, 0) for p in portraits if p.get(field) is not None]
        if values:
            summary['statistics'][field] = {
                'mean': float(np.mean(values)),
                'median': float(np.median(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values)),
                'std': float(np.std(values))
            }
    
    # Статистика по категориальным признакам
    categorical_fields = ['preferred_channel', 'top_category', 'preferred_os']
    for field in categorical_fields:
        values = [p.get(field) for p in portraits if p.get(field) is not None]
        if values:
            from collections import Counter
            summary['statistics'][field] = dict(Counter(values))
    
    # Сохраняем сводку
    import json
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False, default=str)
    
    print(f"📊 Сводная статистика сохранена в {output_file}")
    return summary


# Пример использования пакетной обработки:
# 
# # Выберите несколько пользователей для анализа
# sample_user_ids = [59328774, 66295302, 37542104, 35193548, 27256137]
# 
# # Создайте портреты
# portraits = create_user_portraits_batch(
#     user_ids=sample_user_ids,
#     users_df=datasets['users'],
#     items_dict=items_dict,
#     base_path="./t_ecd_data/dataset/small",
#     save_to_file=True,
#     output_dir="./user_portraits"
# )
# 
# # Создайте сводную статистику
# if portraits:
#     summary = create_portraits_summary(portraits, "portraits_summary.json")
